In [1]:
#IMPORTS
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import preprocessing

## Part 3: VALIDATOR to test
modifier and simulator working in dynamic way, with hardcoded ranges
* next step, extract the ranges 



In [2]:
# # DRAFT TEMP FUNCTIONS that will be in validator once they work as intended
# from sklearn.linear_model import HuberRegressor
# from sklearn.preprocessing import PolynomialFeatures

# def get_range(x_values,outliers,delta=0.7):
#     x_values = np.array(x_values)
#     relative_max = x_values[-1]
#     relative_min = x_values[0]
    
#     # Calculate ranges of outliers
#     outlier_ranges = []

#     current_range = [outliers[0]]
#     print('current range: ',current_range)

#     for i in range(1, len(outliers)):
#         x_outlier_ranges = []

#         if outliers[i] == outliers[i-1] + 1:
#             current_range.append(outliers[i])
#         else:
            
#             outlier_ranges.append(current_range)
#             current_range = [outliers[i]]

#     # Add the last range
#     outlier_ranges.append(current_range)
#     print(outlier_ranges)
#     # Get x value ranges for each outlier range
#     x_range_max=relative_max
#     x_range_min = relative_min
#     for outlier_range in outlier_ranges:
#         print('\noutlier_range ',outlier_range)
#         if np.max(outlier_range)<(np.shape(x_values)[0]-1) and np.min(outlier_range)>0: 
#             delta_low=delta*(x_values[np.min(outlier_range)] - x_values[np.min(outlier_range)-1])
#             delta_high=delta*np.abs(x_values[np.max(outlier_range)] - x_values[np.max(outlier_range)+1])
#             x_range_min = np.min(x_values[outlier_range])-delta_low
#             x_range_max = np.max(x_values[outlier_range])+delta_high
#             print('I am withing the min max')
#         elif x_range_max>relative_max or np.max(outlier_range)==(np.shape(x_values)[0]-1):
#             print('I am at max')
#             delta_low=delta*(x_values[np.min(outlier_range)] - x_values[np.min(outlier_range)-1])
#             x_range_min = np.min(x_values[outlier_range])-delta_low
#             x_range_max=relative_max
#         elif x_range_min<relative_min or np.min(outlier_range)==0:
#             print('I am at min')
#             delta_high=delta*np.abs(x_values[np.max(outlier_range)] - x_values[np.max(outlier_range)+1])
#             x_range_max = np.max(x_values[outlier_range])+delta_high
#             x_range_min = relative_min
#         x_outlier_ranges.append((x_range_min, x_range_max))


#     print("Ranges of outlier x values:")
#     for i, x_range in enumerate(x_outlier_ranges):
#         print(f"Outlier Range {i+1}: {x_range[0]} to {x_range[1]}")
    
#     return x_outlier_ranges



# def curve_fit(mod_x,sim_y, threshold=3):
#     # Example data
#     # mod_x = [1.0, 2.4347826086956523, 6.260869565217392, 12.478260869565217, 21.08695652173913, 32.086956521739125, 45.47826086956522, 61.26086956521739, 79.43478260869566, 100.0]
#     # sim_y = [-18785.46448566434, -22589.28977083527, -13875.22366018544, -8366.169916783167, -10461.393801292468, 5188.58062260236, 76235.86972207189, 197523.09425923007, 435446.98046035116, 1004675.2544642929]

#     # Reshape the arrays to match HuberRegressor's expectations
#     x_values = np.array(mod_x)
#     y_values = np.array(sim_y)

#     # Create polynomial features
#     poly = PolynomialFeatures(degree=2)
#     x_poly = poly.fit_transform(x_values.reshape(-1, 1))

#     # Fit a polynomial model using HuberRegressor
#     huber_regressor = HuberRegressor()
#     huber_regressor.fit(x_poly, y_values)

#     # Predict y values based on the fitted polynomial model
#     predicted_y_values = huber_regressor.predict(x_poly)

#     # Calculate residuals (differences between actual y and predicted y)
#     residuals = y_values - predicted_y_values

#     # Calculate the median absolute deviation (MAD) of the residuals
#     median_absolute_deviation = np.median(np.abs(residuals))

#     # Define a threshold for outliers (e.g., 3 times MAD)
#     outlier_threshold = threshold * median_absolute_deviation

#     # Identify outliers
#     outliers = np.where(np.abs(residuals) > outlier_threshold)[0]

#     # Identify points of good fit
#     good_fit_points = np.where(np.abs(residuals) <= outlier_threshold)[0]

#     print("Outlier indices:", outliers)
#     print("Good fit indices:", good_fit_points)

#     # Plot the original data and the fitted polynomial curve
#     plt.scatter(x_values, y_values, label='Data points')
#     plt.plot(x_values, predicted_y_values, color='red', label='Fitted polynomial')
#     plt.legend()
#     plt.xlabel('X')
#     plt.ylabel('Y')
#     plt.title('Fitted Polynomial and Data Points')
#     plt.show()
#     return outliers, good_fit_points


## PART 4: Main Func: with Modifier.py

In [1]:
## VALIDATOR FILE: :
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
from global_settings import fitting_threshold, mdv


# import global here: threshold

class Validator:
    def __init__(self):
        self.iterations = 1
        self.total_points = 0
        self.total_bad_points = 0
        self.range = (mdv["domain_min_range"], mdv["domain_max_range"])
        self.num_points_evaluated = 0
        self.history = []  # Initialize history as an empty list

    def fit_curve(x_values, y_values, global_range=0):
        x_values = np.array(x_values)  # Convert to numpy array
        y_values = np.array(y_values)  # Convert to numpy array

        # fit model
        model = HuberRegressor(epsilon=1)
        model.fit(x_values.reshape(-1, 1), y_values)

        # Get the coefficients of the fitted line
        slope = model.coef_[0]
        intercept = model.intercept_

        return slope, intercept


    def get_unfitting_point(x_values, y_values, fitted_curve, threshold=0.9):
        # TODO: Apply curve fit to x values, returns y_values
        # TODO: If diff between y_values and true_y_values is greater than threshold, save point as unfit point
        temp_soln = [1, 4, 5, 6]
        unfit_points = [x_values[i] for i in temp_soln], [y_values[i] for i in temp_soln]

        # Returns all unfit y_values at mod_x points
        return unfit_points

    def generate_unfitting_ranges(x_values,sim_y_list,threshold=fitting_threshold):
        # apply curve fit to new data
        fitted_curve = Validator.fit_curve(x_values,sim_y_list)
        # get points of unfit
        unfit_points, fit_points = Validator.thresholding(fitted_curve,threshold)

        # create ranges from continuous unfit points
        if Validator.iterations == 0:
            unfit_ranges = [[0,4],[40,50]]

        Validator.collect_data(sim_y_list,x_values,unfit_ranges,unfit_points=unfit_points)
        # return unfit ranges
        # Validator.collect_data()
        return unfit_ranges

    # def update_history(self, new_total_points, new_range):
    #     self.iterations += 1
    #     self.total_points += new_total_points
    #     self.range = new_range

    # def generate_report(self):
    #     print('\nThe iteration is: ',self.iterations)
    #     print('The range is: ', self.range)
    #     print('The total point count is: ', self.total_points)

    def update_num_points_evaluated(self, points, min_x, max_x):
        self.num_points_evaluated = len([(x, y) for x, y in points if min_x <= x <= max_x])
    
    def local_exploration_validator_A(x_values, y_values, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=fitting_threshold):
        # generate fit function 
        fitted_curve = Validator.fit_curve(x_values, y_values,global_range)
        # find misfit points from mod_x_list and sim_y_list (outliers) using threshold from fit function
        x_values_unfit, y_values_unfit = Validator.get_unfitting_point(x_values, y_values,fitted_curve,threshold = threshold)
        # generate ranges of misfit points (make it fancy threshold)
        unfitting_ranges = Validator.generate_unfitting_ranges(x_values_unfit, y_values_unfit,threshold = threshold)
        ## update your history (for each iteration: 

        # Update history with new values
        Validator.update_history(5, 100, (10, 20))  # For example, 5 new iterations, 100 new total points, and new range
        Validator.generate_report()

        # Update num_points_evaluated (self, new_iterations, new_total_points, new_range)
        Validator.update_num_points_evaluated(len(x_values), min=global_range[0], max=global_range[1])
        return unfitting_ranges

    def validator_controller(mod_x_list,sim_y_list, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=fitting_threshold, local_validator=local_exploration_validator_A, do_plot=False):
        # gets points mod_x_list, sim_y_list
        validator_ranges=local_validator(mod_x_list,sim_y_list, global_range=[mdv["domain_min_range"],mdv["domain_max_range"]],threshold=threshold)

        # if not first time accessing validator, merge old points with new
            # i.e. merge all data points together
        return validator_ranges

    # def collect_data(self, sym, mod, unfit_points):
    #     self.archive_sym.append(sym)
    #     self.archive_mod.append(mod)
    #     self.iterations += 1
    #     self.total_points += len(mod)
    #     self.total_unfit_points += len(unfit_points)
    #     self.history.extend(sym)  # Add sym to history

    # def update_statistics(self, new_sym, new_mod):
    #     self.collect_data(new_sym, new_mod, [])
    #     self.history.extend(new_sym) 
    
    def thresholding(self, threshold):
        # Assuming you want to return x values above the threshold
        x_values = np.arange(len(self.history))
        y_values = np.array(self.history)
        above_threshold = x_values[y_values > threshold]
        return above_threshold

# For the purpose of testing just the validator, let's use this list: 
import numpy as np
from global_settings import mdv
# from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator

# Generate x values
mod_x = np.linspace(-2, 2, 20)

# Generate y values resembling a parabola shape
sim_y_list = mod_x**2 + np.random.normal(0, 0.2, 20)

# Add four outliers
outliers_indices = [6, 8, 11, 14]
outliers_values = np.array([0.8, 1.5, 1.2, 1.6])

sim_y_list[outliers_indices] = outliers_values
print('** Sim y and Mod x output\n', sim_y_list,'\n',mod_x,'\n') 

output = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
print('** Validator output\n', output,'\n\n') 



** Sim y and Mod x output
 [4.2249509  3.14924563 2.16386136 1.42689453 1.11097281 0.71429068
 0.8        0.38876972 1.5        0.06110269 0.22013017 1.2
 0.30137914 0.61461927 1.6        1.27431409 1.87514754 2.61376496
 3.22705875 3.85026519] 
 [-2.         -1.78947368 -1.57894737 -1.36842105 -1.15789474 -0.94736842
 -0.73684211 -0.52631579 -0.31578947 -0.10526316  0.10526316  0.31578947
  0.52631579  0.73684211  0.94736842  1.15789474  1.36842105  1.57894737
  1.78947368  2.        ] 



AttributeError: 'tuple' object has no attribute 'history'

In [10]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.linear_model import HuberRegressor
# from sklearn.preprocessing import StandardScaler
# from global_settings import fitting_threshold, mdv

# # Define the thresholding function
# def thresholding(history, threshold):
#     x_values = np.arange(len(history))
#     y_values = np.array(history)
#     above_threshold = x_values[y_values > threshold]
#     return above_threshold

# class Validator:
#     def __init__(self):
#         self.iterations = 1
#         self.total_points = 0
#         self.total_bad_points = 0
#         self.range = (mdv["domain_min_range"], mdv["domain_max_range"])
#         self.num_points_evaluated = 0
#         self.history = []  # Initialize history as an empty list

#     @staticmethod
#     def fit_curve(x_values, y_values, global_range=0):
#         x_values = np.array(x_values)  # Convert to numpy array
#         y_values = np.array(y_values)  # Convert to numpy array

#         # standardize
#         x_scaler, y_scaler = StandardScaler(), StandardScaler()
#         x_train = x_scaler.fit_transform(x_values.reshape(-1, 1))
#         y_train = y_scaler.fit_transform(y_values.reshape(-1, 1))

#         # fit model
#         model = HuberRegressor(epsilon=1)
#         model.fit(x_train, y_train.ravel())

#         # Get the coefficients of the fitted line
#         slope = model.coef_[0]
#         intercept = model.intercept_

#         return slope, intercept
    

#     def get_unfitting_point(x_values, y_values, fitted_curve, threshold=0.9):
#         # TODO: Apply curve fit to x values, returns y_values
#         # TODO: If diff between y_values and true_y_values is greater than threshold, save point as unfit point
#         temp_soln = [1, 4, 5, 6]
#         unfit_points = [x_values[i] for i in temp_soln], [y_values[i] for i in temp_soln]

#         # Returns all unfit y_values at mod_x points
#         return unfit_points

#     def generate_unfitting_ranges(x_values, sim_y_list, threshold=fitting_threshold):
#         # apply curve fit to new data
#         fitted_curve = Validator.fit_curve(x_values, sim_y_list)
#         # get points of unfit
#         unfit_points, fit_points = thresholding(fitted_curve, threshold)

#         # create ranges from continuous unfit points
#         if Validator.iterations == 0:
#             unfit_ranges = [[0, 4], [40, 50]]

#         Validator.collect_data(sim_y_list, x_values, unfit_ranges, unfit_points=unfit_points)
#         # return unfit ranges
#         # Validator.collect_data()
#         return unfit_ranges

#     # def update_history(self, new_total_points, new_range):
#     #     self.iterations += 1
#     #     self.total_points += new_total_points
#     #     self.range = new_range

#     # def generate_report(self):
#     #     print('\nThe iteration is: ', self.iterations)
#     #     print('The range is: ', self.range)
#     #     print('The total point count is: ', self.total_points)

#     def update_num_points_evaluated(self, points, min_x, max_x):
#         self.num_points_evaluated = len([(x, y) for x, y in points if min_x <= x <= max_x])

#     def local_exploration_validator_A(x_values, y_values, global_range=[mdv["domain_min_range"], mdv["domain_max_range"]],
#                                      threshold=fitting_threshold):
#         # generate fit function
#         fitted_curve = Validator.fit_curve(x_values, y_values, global_range)
#         # find misfit points from mod_x_list and sim_y_list (outliers) using threshold from fit function
#         x_values_unfit, y_values_unfit = Validator.get_unfitting_point(x_values, y_values, fitted_curve, threshold=threshold)
#         # generate ranges of misfit points (make it fancy threshold)
#         unfitting_ranges = Validator.generate_unfitting_ranges(x_values_unfit, y_values_unfit, threshold=threshold)
#         ## update your history (for each iteration:

#         # Update history with new values
#         # Validator.update_history(5, (10, 20))  # For example, 5 new iterations, 100 new total points, and new range
#         # Validator.generate_report()

#         # Update num_points_evaluated (self, new_iterations, new_total_points, new_range)
#         Validator.update_num_points_evaluated(len(x_values), min=global_range[0], max=global_range[1])
#         return unfitting_ranges

#     def validator_controller(mod_x_list, sim_y_list, global_range=[mdv["domain_min_range"], mdv["domain_max_range"]],
#                              threshold=fitting_threshold, local_validator=local_exploration_validator_A, do_plot=False):
#         # gets points mod_x_list, sim_y_list
#         validator_ranges = local_validator(mod_x_list, sim_y_list, global_range=global_range, threshold=threshold)


#     # def collect_data(self, sym, mod, unfit_points):
#     #     self.archive_sym.append(sym)
#     #     self.archive_mod.append(mod)
#     #     self.iterations += 1
#     #     self.total_points += len(mod)
#     #     self.total_unfit_points += len(unfit_points)
#     #     self.history.extend(sym)  # Add sym to history


#     # def update_statistics(self, new_sym, new_mod):
#     #     self.collect_data(new_sym, new_mod, [])
#     #     self.history.extend(new_sym) 

#     def thresholding(history, threshold):
#         x_values = np.arange(len(history))
#         y_values = np.array(history)
#         x_values_t = x_values[y_values > threshold]
#         y_values_t = y_values[y_values > threshold]
#         return x_values_t, y_values_t

#         # if not first time accessing validator, merge old points with new
#         #

# # Create an instance of the Validator class
# # Create an instance of the Validator class
# validator = Validator()

# # Assuming you have x_values and y_values
# x_values = [1, 2, 3, 4, 5]
# y_values = [2, 4, 5, 4, 5]

# # Call the fit_curve function
# slope, intercept = validator.fit_curve(x_values, y_values)
# print("Slope:", slope)
# print("Intercept:", intercept)

# # Call the get_unfitting_point function
# fitted_curve = [1, 3, 5, 7, 9]  # Example fitted_curve for demonstration
# unfit_points = validator.get_unfitting_point(x_values, y_values, fitted_curve)
# print("Unfit Points:", unfit_points)

# # Call the generate_unfitting_ranges function
# x_values = [1, 2, 3, 4, 5]  # Example x_values for demonstration
# sim_y_list = [2, 4, 5, 4, 5]  # Example sim_y_list for demonstration
# unfitting_ranges = validator.generate_unfitting_ranges(x_values, sim_y_list)
# print("Unfitting Ranges:", unfitting_ranges)

# # Call the update_history function
# new_total_points = 10
# new_range = (20, 30)
# validator.update_history(new_total_points, new_range)

# # Call the generate_report function
# validator.generate_report()

# # Call the update_num_points_evaluated function
# points = [(1, 2), (3, 4), (5, 6)]  # Example points for demonstration
# min_x = 0
# max_x = 5
# validator.update_num_points_evaluated(points, min_x, max_x)

# # Call the local_exploration_validator_A function
# global_range = [0, 10]
# threshold = 0.8
# unfitting_ranges = validator.local_exploration_validator_A(x_values, y_values, global_range, threshold)
# print("Unfitting Ranges:", unfitting_ranges)

# # Call the validator_controller function
# mod_x_list = [1, 2, 3, 4, 5]  # Example mod_x_list for demonstration
# sim_y_list = [2, 4, 5, 4, 5]  # Example sim_y_list for demonstration
# threshold = 0.9
# output = validator.validator_controller(mod_x_list, sim_y_list, threshold=threshold)
# print("Validator Output:", output)



In [ ]:
# # PLACEHOLDER FOR THE MAIN FUNCTION V2 DRAFT 1: 
# %load_ext autoreload
# %autoreload 2

# from global_settings import mdv
# # from Validator import Validator
# from Modifier import Modifier
# from Simulator import Simulator

# print('* MDV: \n',mdv,'\n')
# #TODO: How to autoreload without restarting kernel (for global vars)
# # Instantiate mdv values
# # mdv={"domain_min_range":1, "domain_max_range":100, "modifier_incremental_unit":3, "modifier_data_point":10}
# # Create an instance of the Validator Class
# validate = Validator
# modifier = Modifier
# simulator = Simulator
# appendedvars=[]

# mainfunc=True
# # Initialize interval list
# interval_lists=[(mdv["domain_min_range"], mdv["domain_max_range"])]
# x=1
# while mainfunc==True:

#     # Run Modifier Controller Function
#     mod_x_list= modifier.modifier_controller(range_list=interval_lists,local_modifier=modifier.local_modifier_A, do_plot=False)
#     # appendedvars.append(mod_x_list)

#     #if data can't be generated:
#     if mod_x_list == False: # FALSE IF ["modifier_data_point"] < mdv["modifier_incremental_unit"]:
#         print('*   ITERATIONS END HERE   *')
#         break

#     # Run Simulator Function
    
#     # for i in range(np.shape(mod_x_list)[0]):

#     mod_x,sim_y_list = simulator.simulator_controller(mod_x_list,selected_function=simulator.sim_func_A)
#     # print((mod_x))
#     # print((sim_y_list))
#     assert len(mod_x) == len(sim_y_list)
#     # Get ranges of unfit points ( IF ANY )

#     # Run Validator Controller functions
#     output = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
    
        
#     # Example Usage:

#     # TODO fix error in Validator function
#     # TODO: Generate Report
#     # ranges = validate.get_unfitting_ranges(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
#     # print('these are the ranges of validator\n',ranges)

#     ## TODO: Replace for loops with Validator functions that return ranges [(),(),()] format
#     if x == 1: 
#         ranges = [(0,50),(70,80)]
#     elif x == 2:
#         ranges = [(30,50),(70,75)]
#     elif x==3: 
#         ranges = False
#     x=x+1
#     if mod_x_list == False or not ranges: 
#         mainfunc=False
#         print('The END')
#     interval_lists = ranges


# # TODO: 
# # Validator needs to work as a loop within itseft that accepts 3 different sets of points
# # understand curve fitting
# # Fix ranges


NameError: name 'Validator' is not defined

In [14]:
# PLACEHOLDER FOR THE MAIN FUNCTION V2
%load_ext autoreload
%autoreload 2

from global_settings import mdv
# from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator

print('* MDV: \n',mdv,'\n')
#TODO: How to autoreload without restarting kernel (for global vars)
# validate = Validator
modifier = Modifier
simulator = Simulator
appendedvars=[]

mainfunc=True
# Initialize interval list
interval_lists=[(mdv["domain_min_range"], mdv["domain_max_range"])]
x=1
while mainfunc==True:
    mod_x_list= modifier.modifier_controller(range_list=interval_lists,local_modifier=modifier.local_modifier_A, do_plot=False)

    if mod_x_list == False: # FALSE IF ["modifier_data_point"] < mdv["modifier_incremental_unit"]:
        print('*   ITERATIONS END HERE   *')
        break

    mod_x,sim_y_list = simulator.simulator_controller(mod_x_list,selected_function=simulator.sim_func_A)
    assert len(mod_x) == len(sim_y_list)

    output = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
    print('** Validator output\n', output,'\n\n') 
    
    # ranges = Validator.get_unfitting_ranges(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)

    # if x == 1: 
    #     ranges = [(0,50),(70,80)]
    # elif x == 2:
    #     ranges = [(30,50),(70,75)]
    # elif x==3: 
    #     ranges = False
    # x=x+1
    # if mod_x_list == False or not ranges: 
    #     mainfunc=False
    #     print('The END')
    # interval_lists = ranges



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
* MDV: 
 {'domain_min_range': 1, 'domain_max_range': 100, 'modifier_incremental_unit': 3, 'modifier_data_point': 1} 


Modifier controller...
  * Interval:  [(1, 100)]
*   ITERATIONS END HERE   *
